In [1]:
using StanModels

ProjDir = rel_path_s("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_2 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchains.

In [3]:
stanmodel = Stanmodel(name="m12.6.2",  model=m12_6_2,
output_format=:mcmcchains);

Input data for cmdstan

In [4]:
m12_6_2_data = Dict("N" => size(d, 1), "T" => d[:total_tools],
"N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_2_data, ProjDir,
diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Log evidence      = 0.0
Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = alpha, a_society.1, a_society.2, a_society.3, a_society.4, a_society.5, a_society.6, a_society.7, a_society.8, a_society.9, a_society.10, bp, sigma_society, log_lik.1, log_lik.2, log_lik.3, log_lik.4, log_lik.5, log_lik.6, log_lik.7, log_lik.8, log_lik.9, log_lik.10

Empirical Posterior Estimates
──────────────────────────────────────────────────────
parameters
                Mean    SD   Naive SE  MCSE     ESS   
  a_society.1 -0.2106 0.2619   0.0041 0.0100  685.2982
  a_society.2  0.0394 0.2410   0.0038 0.0085  808.9609
  a_society.3 -0.0434 0.2030   0.0032 0.0065  963.7376
  a_society.4  0.3278 0.1972   0.0031 0.0064  939.7793
  a_society.5  0.0439 0.1844   0.0029 0.0054 1000.0000
  a_society.6 -0.3240 0.2194   0.0035 0.0072  917.1036
  a_society.7  0.1462 0.1841   0.0029 0.0048 1000.0000
  a_society.8 -0.1750 0.1905   0.0030 0.0050 10

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*